# lesson6
<hr>
 大家在lesson 4中已经生成得到了Camponotus floridanus蚂蚁全基因组的蛋白序列fasta文件，总共有17064条蛋白序列。现在，有另一种蚂蚁Acromyrmex echinatior全基因组的蛋白序列，路径如下：

Aech_v3.10.geneWithUTR.pep.gz

本次的任务是通过双向比对最好的方法(Reciprocal Best Hit, RBH)鉴定这两种蚂蚁的**直系同源基因(ortholog)**。RBH是鉴定物种间直系同源基因最传统也是最简单的方法，在进化分析中经常用到。（思考：什么是直系同源
基因？什么是旁系同源基因？两者的差别是什么？）

步骤：

1、用Linux的cat命令把两种蚂蚁的蛋白序列fasta文件合并成一个fasta文件(如名字叫merge.pep)；

2、用blast软件做蛋白对蛋白的序列比对，query文件和subject文件都是merge.pep，相当于merge.pep自身比自身。输出格式设置为m8格式。为了提高比对的效率，要求把query文件切成100个子文件，每个子文件同时对subject文件做blast，100个比对任务提交到计算集群并行跑，都完成后把100个m8文件直接cat起来，作为总的m8文件。(思考：这跟直接用query文件比subject文件，结果有什么不一样？)

开始分析前，需要先了解一下blast比对的基本原理以及用法。

3、写一个perl脚本，以m8文件作为输入，通过双向比对最好的原则鉴定两个物种的ortholog pairs。判定一个物种的某基因在另一个物种中的best hit，可以根据blast score值的高低，score越高越好。例如Cflo蚂蚁的A gene，在Aech蚂蚁中比上了10个hit (分别是A、B、C、…...)，通过score判断，best hit是Aech的B基因；Aech的B gene，在Cflo中比上了5个hit，通过score值判断，best hit是Cflo的A基因。则Cflo的A gene和Aech的B gene是一对直系同源基因。要求：尝试在本脚本中使用哈希数组。

## answer
<hr>

#### 直系同源基因和旁系同源基因

![Ortholog paralog](https://upload.wikimedia.org/wikipedia/commons/4/4b/Ortholog_paralog_analog_examples.svg)

举例来说，人类（Human）和黑猩猩（Chimpanzee）的共同祖先中存在Histone H1基因，通过gene duplication之后形成Histone H1.1和Histone H1.2，在进化过程中，这两个基因在人类和黑猩猩中被保留。那么，人类或者黑猩猩的Histone H1.1和Histone H1.2被称为旁系同源(Paralog)，人类的Histone H1.1与黑猩猩的Histone H1.1称为直系同源(Ortholog)。

### 用RBH鉴定Ortholog pairs
<hr>

**步骤如下**：
1. 用Linux的cat命令把两种蚂蚁的蛋白序列fasta文件合并成一个fasta文件(如名字叫merge.pep)：  
`cat Aech_v3.10.geneWithUTR.pep Cflo.pep > merge.pep`

2. 利用blastp进行比对：  
 - **建立database:**  
  `makeblastdb -in merge.pep -dbtype prot`
 - **比对：**  
  `blastp -num_threads 8 -query merge.pep -db merge.pep -out merge.out -evalue 1 -outfmt 6`  

3. 检索Ortholog pairs

**为了提高比对效率，将merge.pep分割为100个文件，每个文件包含的序列大致相同。**，将单个文件如merge.pep_1与merge.pep进行比对，得到merge.pep_1中的每条蛋白序列的hits，所有100个文件得到的out进行合并即可。结果截图：  

![blastp](./image/lesson6_split.JPG)

比对结果截图：

![blastp](./image/lesson6_blastp.JPG)

最后，将合并结果进行分析，利用RBH筛选出两个物种的Ortholog pairs。

In [ ]:
f = open('./data/lesson6/out', 'r')
cur_query = ''
cur_subject = ''
max_score = 0
best_hit = {}
result = []
for line in f:
    line = line.strip().split('\t')
    query_species = line[0].split('_')[0]
    subject_species = line[1].split('_')[0]
    if cur_query:
        if line[0] == cur_query:
            if query_species != subject_species:
                cur_score = float(line[-1])
                if cur_score > max_score:
                    cur_subject = line[1]
                    max_score = cur_score
        else:
            if query_species != subject_species:
                best_hit[cur_query] = cur_subject
                cur_query = line[0]
                cur_subject = line[1]
                max_score = float(line[-1])
    else:
        query_species = line[0].split('_')[0]
        subject_species = line[1].split('_')[0]
        if query_species != subject_species:
            cur_query = line[0]
            cur_subject = line[1]
            max_score = float(line[-1])
best_hit[cur_query] = cur_subject
f.close()

#检索ortholog pairs
pairs = set()
for key, value in best_hit.items():
    try:
        if value not in pairs:
            if best_hit[value] == key:
                result.append((key, value))
                pairs.add(key)
    except KeyError:
        continue

#写入文件
with open('lesson6_result.txt', 'w'):
    w.write('\n'.join([i + '\t' + j for i, j in result]))

结果截图：

![lesson6_result](./image/lesson6_result.JPG)

在blastp比对时，参数设置可能会影响结果。

## Reference
<hr>

[如何本地化进行blast比对](http://blog.sciencenet.cn/blog-223428-616812.html)

[Blast m8 format](http://blog.163.com/wxh244295043@yeah/blog/static/1726580122015515101815737/)

[Sequence homology](https://en.wikipedia.org/wiki/Sequence_homology#Orthology)